In [4]:
import streamlit as st
from pyvis.network import Network
import pandas as pd
import streamlit.components.v1 as components
from db_info import user, password, host, database, port
import mysql.connector
import plotly.express as px
import plotly.graph_objects as go


# MySQL 데이터베이스 연결
db_connection = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database,
    port=port
)

cursor = db_connection.cursor()

# 데이터 가져오기
cursor.execute("SELECT 판례일련번호,사건번호,클러스터,판례내용_이유 FROM inheritance_keyword")
result = cursor.fetchall()

# DataFrame 생성 및 컬럼 이름 설정
df_inheritance = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

In [5]:
df_inheritance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4127 entries, 0 to 4126
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   판례일련번호   4127 non-null   int64 
 1   사건번호     4127 non-null   object
 2   클러스터     4127 non-null   int64 
 3   판례내용_이유  4127 non-null   object
dtypes: int64(2), object(2)
memory usage: 129.1+ KB


In [80]:
df_inheritance.to_csv('저장저장.csv')

In [33]:
import pandas as pd

df_inheritance = pd.read_csv('저장저장.csv')

In [34]:
df_inheritancelist = pd.read_csv('inheritance_list.csv')

In [ ]:
# map()을 사용하여 df_inheritance의 '판례일련번호'에 해당하는 '법원명' 추가
df_inheritance['법원명'] = df_inheritance['판례일련번호'].map(df_inheritancelist.set_index('판례일련번호')['법원명'])

In [37]:
df_inheritance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4127 entries, 0 to 4126
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4127 non-null   int64 
 1   판례일련번호      4127 non-null   int64 
 2   사건번호        4127 non-null   object
 3   클러스터        4127 non-null   int64 
 4   판례내용_이유     4127 non-null   object
 5   법원명         4127 non-null   object
dtypes: int64(3), object(3)
memory usage: 193.6+ KB


In [38]:
import re

# 재판장 이름을 추출하는 함수 (판례일련번호 포함)
def extract_judge_name(row):
    text = row['판례내용_이유']
    case_number = row['판례일련번호']  # 판례일련번호 추출
    law_name = row['법원명']

    
    # 각 패턴에 맞는 정규 표현식 정의
    patterns = [
        r'대법원장\s([^\(]+)\(재판장\)',  # 대법원장 @@@(재판장)
        r'대법관\s([^\(]+)\(재판장\)',   # 대법관 @@@(재판장)
        r'심판관\s([^\(]+)\(심판장\)',   # 심판관 @@@(심판장)
        r'판사\s([^\(]+)\(재판장\)',     # 판사 @@@(재판장)
        r'판사\s([^\(]+)'                # 판사 @@@ (재판장 없음)
    ]
    
    # 각 패턴에 대해 텍스트에서 재판장 이름을 추출
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return case_number,law_name, match.group(1)  # 판례일련번호와 재판장 이름을 튜플로 반환
    
    return case_number,law_name, None  # 패턴이 없으면 판례일련번호와 None 반환

# df_result에 판례일련번호와 재판장 이름 추출
df_result = pd.DataFrame(df_inheritance.apply(extract_judge_name, axis=1).tolist(), columns=['판례일련번호', '법원명','재판장_이름'])

In [39]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4127 entries, 0 to 4126
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판례일련번호  4127 non-null   int64 
 1   법원명     4127 non-null   object
 2   재판장_이름  4089 non-null   object
dtypes: int64(1), object(2)
memory usage: 96.9+ KB


In [43]:
#재판장이름 없는 행 확인
null_row = df_result[df_result['재판장_이름'].isnull()]

In [44]:
#재판장이름 없는 판례일련번호 api확인하기
null_row

,판례일련번호,법원명,재판장_이름
303,201010,대법원,None
304,201132,대법원,None
305,201211,대법원,None
335,107950,대법원,None
336,107948,대법원,None
1196,170962,대법원,None
1284,167325,대법원,None
1289,167363,대법원,None
2785,200973,대법원,None
2786,124504,대법원,None


In [ ]:
#판사명 없는 데이터 제외
df_result['재판장_이름'].isnull().sum()

np.int64(38)

In [47]:
frequency = df_result.groupby(['법원명', '재판장_이름']).size().reset_index(name='빈도수')

# 빈도수 기준으로 내림차순 정렬
frequency_sorted = frequency.sort_values(by='빈도수', ascending=False)

# 결과 출력
print(frequency_sorted)

             법원명 재판장_이름  빈도수
230          대법원    박만호   63
275          대법원     윤관   42
235          대법원    박우동   39
292          대법원    이용훈   39
239          대법원    박준서   38
...          ...    ...  ...
1321  춘천지방법원원주지원   김형연     1
1322  춘천지방법원원주지원   정헌명     1
1323        특허법원    김우수    1
1324        특허법원    배기열    1
1325        특허법원    서승렬    1

[1330 rows x 3 columns]


In [48]:
df_result['법원명'].value_counts()

법원명
대법원           2253
서울고등법원         646
대구고등법원         151
서울중앙지방법원       131
서울행정법원          92
              ... 
서울지방법원서부지원       1
청주지방법원충주지원       1
광주지방법원순천지원       1
서울형사지방법원         1
마산지방법원           1
Name: count, Length: 80, dtype: int64

In [49]:
df_result['재판장_이름'].value_counts()

재판장_이름
박만호    67
이용훈    44
김상원    42
윤관     42
이회창    41
       ..
이수욱     1
임기호     1
고재호     1
조진만     1
김택현     1
Name: count, Length: 1035, dtype: int64

In [50]:
a=df_result[df_result['재판장_이름'] == '박만호']
a

,판례일련번호,법원명,재판장_이름
245,115228,대법원,박만호
269,198815,대법원,박만호
280,199718,대법원,박만호
282,110975,대법원,박만호
292,201485,대법원,박만호
...,...,...,...
2924,109292,대법원,박만호
3391,75600,서울고등법원,박만호
3472,75374,서울고등법원,박만호
3477,75364,서울고등법원,박만호
